In [2]:
import pandas as pd
import numpy as np

In [9]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression

def dper_impute(df):
    # Identify missing values in the input DataFrame
    missing_mask = df.isna().values
    
    # Calculate means and covariances for each pair of observed variables
    obs_data = df.loc[~missing_mask].values
    obs_mean = np.mean(obs_data, axis=0)
    obs_cov = np.cov(obs_data, rowvar=False)
    
    # Use the EM algorithm to estimate parameters for the multivariate normal distribution
    n_samples, n_features = df.shape
    if n_features < 6:
        # Handle the case where the input DataFrame has fewer than six columns
        exp_mean = obs_mean.copy()
        exp_cov = obs_cov.copy()
    else:
        while True:
            exp_mean = obs_mean.copy()
            exp_cov = obs_cov.copy()
            for i in range(n_samples):
                for j in range(n_features):
                    if missing_mask[i, j]:
                        exp_mean[j] += (obs_cov[j, j] / obs_cov[i, i]) * (df.iloc[i, :] - obs_mean).values[i] / obs_cov[i, j]
                        exp_cov[:, j] += (obs_cov[j, j] / obs_cov[i, i]) * obs_cov[:, i]
                        exp_cov[j, :] += (obs_cov[j, j] / obs_cov[i, i]) * obs_cov[i, :]
                        exp_cov[j, j] += (obs_cov[j, j] / obs_cov[i, i]) * obs_cov[i, i] / obs_cov[i, j]
            if np.allclose(exp_mean, obs_mean, rtol=1e-4) and np.allclose(exp_cov, obs_cov, rtol=1e-4):
                break
            obs_mean = exp_mean
            obs_cov = exp_cov
    
    # Use regression modeling to impute missing values
    for j in range(n_features):
        if np.any(missing_mask[:, j]):
            X = df.iloc[:, np.arange(n_features) != j].values
            y = df.iloc[:, j].values
            clf = LinearRegression()
            clf.fit(X[~missing_mask[:, j]], y[~missing_mask[:, j]])
            y_pred = clf.predict(X[missing_mask[:, j]])
            df.iloc[missing_mask[:, j], j] = y_pred
    
    return df


In [10]:
from sklearn.datasets import load_iris

data = load_iris()
import pandas as pd
import numpy as np

data1 = pd.DataFrame(data= np.c_[data['data'], data['target']],
                     columns= data['feature_names'] + ['target'])
import pandas as pd
import numpy as np

def drop_random_data(df, column, n, distribution='uniform', **kwargs):
    np.random.seed(123)
    #'uniform', 'standard', 'normal', or 'poisson'. ? 
    if distribution == 'uniform':
        low = 0
        high = df.shape[0] - 1
        indices = np.random.randint(low, high, size=n)

    elif distribution == 'standard':
        # Use a standard normal distribution to generate random indices
        mean = df[column].mean()
        std = df[column].std()

        indices = np.random.normal(loc=mean, scale=std, size=n)
        # Round the indices to the nearest integer

        indices = np.rint(indices).astype(int)

        # Clip the indices to be within the range of valid indices
        indices = np.clip(indices, 0, df.shape[0] - 1)

    else:
        raise ValueError("Invalid distribution: must be 'uniform' or 'standard'.")

    # Set the specified column values to NaN at the selected indices
    df.at[indices, column] = np.nan

    return df

def apply_random_missing_for_all(df,n):
    for each in df.columns:
        df = drop_random_data(df, each, n, distribution='standard')
    return df
new_df = apply_random_missing_for_all(data1,10)
from sklearn.preprocessing import MinMaxScaler

def apply_normalize(df):
    scaler = MinMaxScaler()
    for each in df.columns:
        df[f'{each}'] = scaler.fit_transform(df[[each]])
    return df
n_df = apply_normalize(new_df)
# n_df

In [11]:
n_df

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,0.222222,0.625000,NaN,NaN,NaN
1,0.166667,0.416667,NaN,NaN,NaN
2,0.111111,NaN,NaN,NaN,NaN
3,0.083333,NaN,NaN,0.041667,0.0
4,NaN,NaN,NaN,0.041667,0.0
...,...,...,...,...,...
145,0.666667,0.416667,0.711864,0.916667,1.0
146,0.555556,0.208333,0.677966,0.750000,1.0
147,0.611111,0.416667,0.711864,0.791667,1.0
148,0.527778,0.583333,0.745763,0.916667,1.0


In [12]:
n_df_dper = dper_impute(n_df)
n_df_dper

ValueError: Input X contains NaN.
LinearRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values